In [ ]:
!pip install ultralytics opencv-python

In [ ]:
import cv2
import os
from ultralytics import YOLO
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def draw_pose(img, keypoints, track_id=None):
    """Draw pose keypoints and connections with optional track ID"""
    connections = [(0,1), (0,2), (1,3), (2,4), (5,6), (5,7), (7,9), 
                   (6,8), (8,10), (5,11), (6,12), (11,12), (11,13), 
                   (13,15), (12,14), (14,16)]
    
    # Draw keypoints
    for x, y, conf in keypoints:
        if conf > 0.5:
            cv2.circle(img, (int(x), int(y)), 4, (0, 255, 0), -1)
    
    # Draw connections
    for start, end in connections:
        if (keypoints[start][2] > 0.5 and keypoints[end][2] > 0.5):
            pt1 = (int(keypoints[start][0]), int(keypoints[start][1]))
            pt2 = (int(keypoints[end][0]), int(keypoints[end][1]))
            cv2.line(img, pt1, pt2, (255, 0, 0), 2)
    
    # Draw track ID 
    if track_id is not None and len(keypoints) > 0 and keypoints[0][2] > 0.5:
        head_x, head_y = int(keypoints[0][0]), int(keypoints[0][1])
        cv2.putText(img, f'ID:{track_id}', (head_x, head_y-15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

video_path = "/kaggle/input/360p-video/you_video360p/SSYouTube.online_3 Years - Talks well enough for strangers to understand most of the time_360p.mp4"  
output_folder = "/kaggle/working/"
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_path = os.path.join(output_folder, f"pose_bytetrack{os.path.basename(video_path)}")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
frame_count = 0

model = YOLO('yolov8n-pose.pt').to(device)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")
    
    for result in results:
        if result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()
            confidences = result.keypoints.conf.cpu().numpy()
            
            track_ids = None
            if result.boxes is not None and result.boxes.id is not None:
                track_ids = result.boxes.id.cpu().numpy()
            
            for i, kpts in enumerate(keypoints):
                pose_data = [[kpts[j][0], kpts[j][1], confidences[i][j]] 
                           for j in range(len(kpts))]
                
                current_track_id = int(track_ids[i]) if track_ids is not None and i < len(track_ids) else None
                
                draw_pose(frame, pose_data, current_track_id)
    
    out.write(frame)

cap.release()
out.release()
print(f"Saved {output_path}")